In [0]:
%sql
USE CATALOG content


In [0]:
%sql
DROP TABLE target.silver_advertisements

In [0]:
%sql
SELECT * FROM target.bronze_advertisements

In [0]:
%sql
SELECT * FROM target.silver_account_user_clean

In [0]:
%sql
SELECT * FROM target.silver_follow

In [0]:
%sql
SELECT * FROM target.time

In [0]:
%sql
SELECT t.year,  f.followed_account_id, f.follower_account_id FROM target.silver_follow f
INNER JOIN target.time t ON f.followed_at_time_id = t.time_id
WHERE f.status = "active"
--GROUP BY t.year, f.followed_account_id
ORDER BY 1,2

In [0]:
%sql
-- SUMA NARASTAJĄCA
SELECT DISTINCT t.year,  f.followed_account_id, COUNT(f.follower_account_id) OVER (PARTITION BY f.followed_account_id ORDER BY t.year)
FROM target.silver_follow f
INNER JOIN target.time t ON f.followed_at_time_id = t.time_id
WHERE f.status = "active" 
--GROUP BY t.year, f.followed_account_id
ORDER BY 1 ASC, 3 DESC

In [0]:
%sql
-- GOLDEN 1 -> TOP 1 z kazdego roku pod wzgledem zdobytych nowych obserwujacych
SELECT cte.year, cte.user, cte.number_of_followers FROM (
SELECT DISTINCT t.year,  f.followed_account_id user, COUNT(f.follower_account_id) number_of_followers, row_number() OVER (PARTITION BY t.year ORDER BY COUNT(f.follower_account_id) DESC) row_number
FROM target.silver_follow f
INNER JOIN target.time t ON f.followed_at_time_id = t.time_id
WHERE f.status = "active" 
--AND t.year IN (2019,2020)
GROUP BY t.year, f.followed_account_id
) cte
WHERE row_number = 1

In [0]:
%sql
-- GOLD 2 Podzial wydatkow reklamodawcow na reklamy w kwartałach Q1, Q2, Q3, Q4

SELECT adv.* FROM target.silver_advertisements adv
LEFT JOIN target.silver_advertisements ad ON adv.advertiser_id = ad.advertiser_id